# example code for checking installation

In [2]:
import pyomo.environ as pyo
from pyomo2h5 import save_h5

# Create Model
m = pyo.ConcreteModel()

m.Scenarios = pyo.RangeSet(3)

m.I = pyo.RangeSet(5)

m.p = pyo.Param(initialize=5,doc="this is a parameter")

m.x = pyo.Var(m.I, within=pyo.NonNegativeReals,doc="this is an indexed variable")

@m.Objective(doc="This is an Objective", sense=pyo.minimize)
def obj(m):
    return sum(m.x[i] for i in m.I)


@m.Block()
def scenario(block):
    block.Scenarios = pyo.RangeSet(5)
    block.x = pyo.Var(block.Scenarios, within=pyo.NonNegativeIntegers)
    @block.Constraint()
    def block_con1(block):
        return sum(block.x[j] for j in block.Scenarios) >= 34

@m.Constraint(m.I, doc="This is a Constraint")
def con1(m, i):
    return m.x[i] >= m.scenario.x[i]/10


# Solve Model
filename = "test"

solver = pyo.SolverFactory("gurobi", solver_io="python")
solver.options["LogFile"] = filename + ".log"
results = solver.solve(m, tee=True)

# Save the dictionary to an HDF5 file
save_h5(m, results, filename,
        save_log_flag=False,
        save_constraint_flag=True,
        pickle_flag=True,
        git_flag=True)
print("Dictionary saved to HDF5 file successfully!")

Set parameter OutputFlag to value 1
Set parameter LogFile to value "test.log"
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: AMD Ryzen 7 PRO 5850U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6 rows, 10 columns and 15 nonzeros
Model fingerprint: 0x2fa41dfb
Variable types: 5 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 3e+01]
Presolve removed 6 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 3.4 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.400000000000e+00, best bound 3.400000000000e+00, gap 0.0000%
Dictio